In [8]:
# 4. Automatic Tracing via Environment (Groq Equivalent of wrap_openai)

def simple_groq_call(user_input: str):
    messages = [
        {"role": "system", "content": "You are a concise summarizer."},
        {"role": "user", "content": user_input}
    ]
    # This LLM call is automatically traced because LANGSMITH_TRACING=true is set in .env
    response = llm_client.invoke(messages) 
    return response.content

print("\n--- Automatic Tracing Example (No Decorator) ---\n")
question_auto = "Summarize the key difference between @traceable and the with trace() context manager."
ai_answer_auto = simple_groq_call(question_auto)
print(ai_answer_auto)


--- Automatic Tracing Example (No Decorator) ---

`@traceable` and the `with trace()` context manager are both used for tracing and debugging in Python, particularly with the `logging` module. 

The key difference is:
- `@traceable` is a decorator that enables tracing for a specific function.
- The `with trace()` context manager enables tracing for a specific block of code.


In [7]:
# 3. Execution of Context Manager Example

question_cm = "How do I trace complex logic using the LangSmith Context Manager?"
ai_answer_cm = langsmith_rag(question_cm)

print("--- Question (Context Manager) ---\n")
print(question_cm)
print("\n--- Answer ---\n")
print(ai_answer_cm)

--- Question (Context Manager) ---

How do I trace complex logic using the LangSmith Context Manager?

--- Answer ---

I don't know how to trace complex logic using the LangSmith Context Manager, as the provided context only discusses the @traceable decorator. The context does not mention the LangSmith Context Manager. It only explains how to use the @traceable decorator to log traces and add metadata.


In [6]:
# 2. RAG Functions: Implementing the `with trace()` Context Manager Tweak

@traceable(run_type="retriever", name="Retrieve Documents")
def retrieve_documents(question: str):
    return retriever.invoke(question)   

# TWEAK: Removed @traceable and replaced it with a manual trace context
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)

    # START OF TWEAK: Manual trace context manager implementation
    with trace(
        name="Generate Response (Manual Context)",
        run_type="chain", 
        inputs={"question": question, "documents": documents},
        tags=["manual-context-manager", "lesson-3-tweak"] # Custom tags for identification
    ) as ls_trace:
        messages = [
            {"role": "system", "content": RAG_SYSTEM_PROMPT},
            {"role": "user", "content": f"Context: {formatted_docs} \n\n Question: {question}"}
        ]
        
        # Call the LLM (This nested call is automatically traced by Groq's LangChain wrapper)
        response = llm_client.invoke(messages)
        
        # END OF TWEAK: Manually end the trace and write outputs
        ls_trace.end(outputs={"output": response.content}) 
        
        return response


@traceable(run_type="chain", name="Root RAG Pipeline")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.content

In [5]:
# 1. Setup Environment and Imports (Groq Ready)
import os
from dotenv import load_dotenv
from langsmith import traceable, trace # trace is the context manager
from langsmith.wrappers import wrap_openai # Needed for Groq client equivalence
from langchain_groq import ChatGroq 
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever, RAG_SYSTEM_PROMPT

# --- Configuration ---
load_dotenv()
MODEL_NAME = "llama-3.3-70b-versatile" # Using the confirmed, working Groq model
llm_client = ChatGroq(model=MODEL_NAME)
nest_asyncio.apply()
retriever = get_vector_db_retriever()